In [ ]:
# Libraries
import pandas as pd
from pandas import DataFrame
from nltk import edit_distance
from fuzzywuzzy import fuzz
from datetime import datetime

In [ ]:
# Get data
df = pd.read_csv("cust10k.csv", delimiter="|")

In [ ]:
df.shape

In [ ]:
df.head(10)

In [ ]:
# Drop LOWE and HOME DEPOT
df = df.loc[~((df["gold_site_name"].str.contains("LOWE"))|(df["gold_site_name"].str.contains("HOME DEPOT")))]

In [ ]:
def setup():
    # Add a column to store grouping
    df["GroupA"] = None
    df["GroupB"] = None
    df["GroupC"] = None
    df["GroupA_score"] = None
    df["GroupB_score"] = None
    df["GroupC_score"] = None

In [ ]:
setup() # RUN THIS FIRST!!!

In [ ]:

groups = ["GroupA", "GroupB", "GroupC"]

for g in groups:
    print(f"Starting: {g}")
    print("=" * 72)
    initial = datetime.now()
    df = df.sample(frac=1)
    df = df.reset_index(drop=True)
    # indices of names for looping
    list_idx = [idx for idx in df.index]
    
    # Loop through names
    num_records = 1000 # restrict for debug
    for idx in list_idx[:num_records]: # restrict for debug
    
    # for idx in list_idx:
        print("-" * 72)
        current_name = df.iloc[idx]['gold_site_name']
        print(f"idx:{idx} name to compare: {current_name}")
#         print(df.iloc[idx][g] is None) # for debug
        if df.iloc[idx][g] is None:
            idx_to_compare = list_idx[idx+1:]
#             print(f"Number of names to compare against: {len(idx_to_compare)}")
            for i in idx_to_compare:
                if df.iloc[i][g] is None:
                    compare_name = df.iloc[i]["gold_site_name"]
                    similarity_score = fuzz.ratio(current_name, compare_name)
                    if similarity_score > 80:
    #                     print(f"Similarity score:{similarity_score}")
                        df.loc[df.index[i]][g] = current_name
                        df.loc[df.index[i]][g+"_score"] = similarity_score
        else:
            print(f"{current_name} already matched. Skipping.")
    final = datetime.now()
    total_time = final - initial
    print("=" * 72)
    print(f"Group {g} total time: {total_time}")

print("Saving groups...")
df.to_csv("edit_dist_dedup_groups.csv")
print("Save complete!")

In [ ]:
df.sample(5)

In [ ]:
# df.head()
df.loc[df["GroupA_score"]<100]

In [ ]:
df.shape

In [ ]:
group = "GroupC"
print(f"Number of matches: {df[group].value_counts().sum()}")
print(f"Number of groups: {len(df[group].unique())}")
df[group].value_counts()

In [ ]:
df["GroupA_score"].value_counts()

In [ ]:
df.loc[df["GroupA_score"]==84]

In [ ]:
df.loc[~((df["gold_site_name"].str.contains("LOWE"))|(df["gold_site_name"].str.contains("HOME DEPOT")))]